# Data Visualization with Plotly

The following notebook displays how the plotly suite may be used to visualize data using Python.

## Setting up

Before getting started make sure your python environment is set up correctly, we recommend using `virtualenv` (which also requires `pipx`) for maximum isolation of dependencies.

Restore the dependencies to run this project by running `pip install -r requirements.txt`.

## First hello world with Ploty

The code belows renders a very simple bar chart by using Plotly. Think of it as the basic "hello, world" that confirms your environment is indeed working as expected.

In [1]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

## Getting simple data from an API and visualizing it.

For this example I`ll use [FIPE API](https://deividfortuna.github.io/fipe/) as the source of our data.

The following snippet creates a class to describe a vehicle from this API, calls the API, parses the results into the objects of the Vehicle class and then plots information about the most expensive ones.

### Representing Vehicles using Object orientation

To follow best practices and make things more readable long term we'll be creating Classes to represent the objects that are returned from the API as well as an object to represent the API itself (and thus allow us to have an easier time calling it).

Since we are on an iterative notebook we'll be using the `assert` keyword to test things instead of a proper unit test. In production code we would have an accompanying unit test suite to cover the data contract and any API logic that'd be needed.

Also we'll be using the "type hinting" system from Python's latest versions to further document the objects we'll be creating and working with once we call the api.

In [21]:
import json #since we are working with json data
from typing import Self #for type hinting

# A representation of a Make
class Make:
    def __init__(self: Self, codigo: str, nome: str):
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data) -> Self:
        return cls(**json_data)

# A car Model from a Make.
class Model:
    def __init__(self: Self, codigo: int, nome: str):
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data):
        return cls(**json_data)

# A year of a particular Model
class ModelYear:
    def __init__(self, codigo, nome):
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data):
        return cls(**json_data)

# The price of a Model at a specific year
class ModelYearPrice:
    def __init__(self, TipoVeiculo, Valor, Marca, Modelo, AnoModelo, Combustivel, CodigoFipe, MesReferencia, SiglaCombustivel):
        self.TipoVeiculo = TipoVeiculo
        self.Valor = Valor
        self.Marca = Marca
        self.Modelo = Modelo
        self.AnoModelo = AnoModelo
        self.Combustivel = Combustivel
        self.CodigoFipe = CodigoFipe
        self.MesReferencia = MesReferencia
        self.SiglaCombustivel = SiglaCombustivel

    @classmethod
    def from_json(cls, json_data):
        return cls(**json_data)

# testing if the make class is accurate
makeJson = "{\"codigo\":\"1\",\"nome\":\"Acura\"}"
makeData = json.loads(makeJson)
makeTest = Make.from_json(makeData)
assert makeTest.codigo == "1"
assert makeTest.nome == "Acura"

# testing if the model class is accurate
modelJson = "{\"codigo\":9985,\"nome\":\"Corolla Altis Prem. 1.8 Aut. (HÃ­brido)\"}" 
modelData = json.loads(modelJson)
modelTest = Model.from_json(modelData)
assert modelTest.codigo == 9985
assert modelTest.nome == "Corolla Altis Prem. 1.8 Aut. (HÃ­brido)"

# testing it the modelyear class is accurate
modelYearJson = "{\"codigo\":\"32000-1\",\"nome\":\"32000 Gasolina\"}"
modelYearData = json.loads(modelYearJson)
modelYearTest = ModelYear.from_json(modelYearData)
assert modelYearTest.codigo == "32000-1"
assert modelYearTest.nome == "32000 Gasolina"

# testing if the ModelYearPrice class is accurate
modelYearPriceJson = "{\"TipoVeiculo\":1,\"Valor\":\"R$ 140.970,00\",\"Marca\":\"Toyota\",\"Modelo\":\"Corolla Altis Prem. 1.8 Aut. (HÃ­brido)\",\"AnoModelo\":2022,\"Combustivel\":\"Gasolina\",\"CodigoFipe\":\"002183-0\",\"MesReferencia\":\"novembro de 2024\",\"SiglaCombustivel\":\"G\"}"
modelYearPriceData = json.loads(modelYearPriceJson)
modelYearPriceTest = ModelYearPrice.from_json(modelYearPriceData)
assert modelYearPriceTest.TipoVeiculo == 1
assert modelYearPriceTest.Valor == "R$ 140.970,00"
# not testing the rest of the attributes since they are the same as the ones in the json

### Building a Fluent class to Query the API

To make it easier to interact with the remote API we'll build a "Builder" class that handles the additional logic of building out the API endpoints following its logic:

1. the base URL must be called in order to get a make's id
2. the models endpoint must be called with the make's id in its path to get the available models
3. the years endpoint must be called with the make's id and the model's id in order to return the yearly entries for that model
4. finally the year's code is used to retrieve the actual price of the vehicle


In [22]:
# BrasilAPI endpoint with all car brands
makeEndpoint = "https://deividfortuna.github.io/fipe/"
modelsEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos"
yearsEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos/<model>/anos"
priceEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos/<model>/anos/<year>"

# TODO; WIP